# MDTraj NetCDF Reporter

Tested through OpenMM reporter interface.

See OpenMM issue: https://github.com/openmm/openmm/issues/4298

See MDTraj issue: https://github.com/mdtraj/mdtraj/issues/1831

Sample system taken from OpenMM Cookbook: https://openmm.github.io/openmm-cookbook/latest/notebooks/tutorials/protein_in_water.html

In [1]:
!wget https://files.rcsb.org/download/1AKI.pdb

--2024-04-18 16:40:38--  https://files.rcsb.org/download/1AKI.pdb
Resolving files.rcsb.org (files.rcsb.org)... 128.6.159.100
Connecting to files.rcsb.org (files.rcsb.org)|128.6.159.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘1AKI.pdb.9’

1AKI.pdb.9              [ <=>                ] 113.59K  --.-KB/s    in 0.1s    

2024-04-18 16:40:38 (1.11 MB/s) - ‘1AKI.pdb.9’ saved [116316]



In [2]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

pdb = PDBFile("1AKI.pdb")

In [3]:
# Specify the forcefield
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

In [4]:
modeller = Modeller(pdb.topology, pdb.positions)
modeller.deleteWater()
residues=modeller.addHydrogens(forcefield)

In [5]:
modeller.addSolvent(forcefield, padding=1.0*nanometer)

In [6]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

In [7]:
#print("Minimizing energy")
simulation.minimizeEnergy()

## Use `NetCDF` reporter

This simulation + reporter set is run in an environment with `netCDF4` installed.

In [8]:
from mdtraj.reporters import NetCDFReporter

In [9]:
reporter = NetCDFReporter('trajectory_netcdf4.nc', 1000)
print(type(reporter._traj_file._handle))

<class 'netCDF4._netCDF4.Dataset'>


In [10]:
simulation.reporters.append(reporter)
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True))
simulation.reporters.append(StateDataReporter("md_log.txt", 100, step=True,
        potentialEnergy=True, temperature=True, volume=True))

print("Running NVT")
simulation.step(2000)

# clear reporters
simulation.reporters.clear()

Running NVT
#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)"
1000,-360806.8896783618,289.72951536530525,236.97179878134529
2000,-357138.6416314868,300.1759240889786,236.97179878134529


## Patch to exclude `netCDF4` for behavior test

This emulates an environment where netCDF4 is not installed and the SciPy fallback is used.

In [11]:
import sys

In [12]:
# unload netCDF4 and mdtraj

sys.modules["netCDF4"] = None

if 'mdtraj.reporters.NetCDFReporter' in sys.modules:
    del sys.modules['mdtraj.reporters.NetCDFReporter']

if 'mdtraj.reporters' in sys.modules:
    del sys.modules['mdtraj.reporters']

if 'mdtraj' in sys.modules:
    del sys.modules['mdtraj']

In [13]:
from mdtraj.reporters import NetCDFReporter

In [14]:
reporter = NetCDFReporter('trajectory_netcdf4.nc', 1000)
print(type(reporter._traj_file._handle))

<class 'scipy.io._netcdf.netcdf_file'>



#################################################################################

The code at netcdf.py:130 requires the netcdf4-python module,
which is a python interface to the NetCDF software libraries and self-describing,
machine-independent data formats that support the creation, access, and
sharing of array-oriented scientific data.

netcdf4-python can be downloaded from https://pypi.python.org/pypi/netCDF,
or installed with the python "coonda" or "pip" package managers using:

# conda install -c conda-forge netCDF4
or
# pip install netCDF4

netcdf4-python also depends on the C-language HDF5 and NetCDF libraries.
For detailed installation instructions, visit
https://unidata.github.io/netcdf4-python/#quick-install

#################################################################################
/home/janash/miniconda3/envs/mdtraj/lib/python3.11/site-packages/mdtraj/formats/netcdf.py:147: UserWarning: Could not find netCDF4 module. Falling back on scipy implementation, which can

In [15]:
simulation.reporters.append(reporter)
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True))
simulation.reporters.append(StateDataReporter("md_log.txt", 100, step=True,
        potentialEnergy=True, temperature=True, volume=True))

In [16]:
print("Running NVT")
simulation.step(2000)

Running NVT
#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)"
3000,-356971.4092096118,302.2788026492188,236.97179878134529
4000,-355657.9013971118,300.679841138615,236.97179878134529
